#### CTD$^2$ Beat AML Challenge
##### Subchallenge 1
_Data files._ 
* Genetic: rnaseq.csv, dnaseq.csv 
* Clinical: clinical_numerical.csv, clinical_categorical.csv, clinical_categorical_legend.csv 
* Response: aucs.csv, response.csv 

_Goal._ Predict AUC from the given data.

_Steps._ 
* Read RNA-seq, DNA-seq, AUC datasets 
* Used DNA-seq for feature selection on RNA-seq 
* Model AUC using RNA-seq 



In [1]:
import os
# import sys
import numpy as np
import scipy as sp
import pandas as pd
# import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr, spearmanr


## Path & Filename...
PATH = os.path.join(os.getenv("HOMEPATH"), "Google Drive\\Study\\ECE 5332-009 - Topics in EE, Data Science\\BeatAML\\")
os.chdir(PATH)

## Read data...
RNA = pd.read_csv(PATH + "rnaseq.csv", header = 0)
DNA = pd.read_csv(PATH + "dnaseq.csv", header = 0)
AUC = pd.read_csv(PATH + "aucs.csv",   header = 0)
# RESP = pd.read_csv(PATH + "response.csv", header = 0)

# print("Dataset sizes = \n", pd.DataFrame([RNA.shape, DNA.shape, AUC.shape], index = ["RNA", "DNA", "AUC"], columns = ["row", "col"]))


In [2]:
## Feature selection...
variant_count = pd.DataFrame([[vv, (DNA.Hugo_Symbol.to_numpy() == vv).sum()] for vv in DNA.Hugo_Symbol.unique()], columns = ["gene", "counts"])
variant_count = variant_count.iloc[variant_count.counts.argsort()[::-1], :].reset_index(drop = True)
# print("#genes with somatic variants = %d\#genes with somatic variant count > 1 = %d" % (variant_count.shape[0], (variant_count.count > 1).sum()))
# print(variant_count)

all([(gg == RNA.Symbol).sum() > 0 for gg in variant_count.gene])    # Check if all genes are in RNA-seq
variant_genes, idx_RNA, _ = np.intersect1d(RNA.Symbol, variant_count.gene, assume_unique = False, return_indices = True)

## Drug information...
drug_info, drug_sample_count = [ ], [ ]
for dd in AUC.inhibitor.unique():
    drug_info.append((dd, AUC.iloc[(AUC.inhibitor == dd).to_numpy(), :]))
    drug_sample_count.append([dd, (AUC.inhibitor == dd).sum()])
drug_info, drug_sample_count = dict(drug_info), pd.DataFrame(drug_sample_count, columns = ["inhibitor", "lab_id_count"])
# print("#inhibitors used = %d\n" % len(drug_info), drug_sample_count)


In [93]:
## Function definitions...
def RF(X_train, y_train, X_test, seed = None):
    mdl = RandomForestRegressor(n_estimators = 100, criterion = 'mse', random_state = seed)
    mdl.fit(X_train, y_train);     y_pred = mdl.predict(X_test)
    return y_pred

def SVM(X_train, y_train, X_test, seed):
    mdl = SVR(kernel = "rbf", degree = 3, gamma = "auto", tol = 1e-3)
    mdl.fit(X_train, y_train);     y_pred = mdl.predict(X_test)
    return y_pred

def PerfEval(y, y_hat, corr = "pearson", err = "NRMSE", alpha = 0.05, pval = False):
    y, y_hat = np.array(y, dtype = float), np.array(y_hat, dtype = float)
    y, y_hat = y.squeeze(), y_hat.squeeze()
    if corr == "pearson":
        CORR, p_val = pearsonr(y, y_hat)
    elif corr == "spearman":
        CORR, p_val = spearmanr(y, y_hat)
    else:
        print('Not valid! Use "pearson" or "spearman" for corr option.')
    ####
#     CORR, p_val = CORR[0], p_val[0]
       
    if p_val > alpha:
        print("No significant correlation!");    CORR = 0
    ####    
    
    if err == "NRMSE":
        ERR = sp.sqrt(((y - y_hat)**2).mean()) / y.std()
    elif err == "RMSE":
        ERR = sp.sqrt(((y - y_hat)**2).mean())
    elif err == "MSE":
        ERR = ((y - y_hat)**2).mean()
    else:
        print('Not valid! Use "NRMSE", "RMSE" or "MSE" for err option.')
    ####
    
    return (CORR, ERR, p_val) if pval else (CORR, ERR)
####


In [102]:
## Pick drug and build dataset for modeling...
drug_picked = list(drug_info.keys())[0];    sample_ids = drug_info[drug_picked]["lab_id"]
# print("Drug picked for analysis = %s\n#samples = %d" % (drug_picked, len(sample_ids)))

X_data = pd.DataFrame(RNA.loc[idx_RNA, sample_ids].to_numpy().T, index = sample_ids, columns = variant_genes)
y_data = pd.DataFrame(drug_info[drug_picked]["auc"].to_numpy(), index = sample_ids, columns = ["auc"])
# print("Analysis dataset size = \n", pd.DataFrame([X_data.shape, y_data.shape], index = ["X", "y"], columns = ["row", "col"]))

## Perform prediction...
CV = KFold(n_splits = 5, shuffle = False, random_state = None)
Y_data_pred = pd.DataFrame(index = sample_ids, columns = ["RF", "SVM"], dtype = float)
for train_idx, test_idx in CV.split(X_data):
    X_train, y_train = X_data.iloc[train_idx, :], y_data.iloc[train_idx]
    X_test,  y_test  = X_data.iloc[test_idx, :],  y_data.iloc[test_idx]
    
    Y_data_pred["RF"][test_idx]  =  RF(X_train, y_train, X_test, seed = None)
    Y_data_pred["SVM"][test_idx] = SVM(X_train, y_train, X_test, seed = None)
####

PCC1, NRMSE1, pval1 = PerfEval(y_data, Y_data_pred["RF"],  pval = True)
PCC2, NRMSE2, pval2 = PerfEval(y_data, Y_data_pred["SVM"], pval = True)

# PERF = pd.DataFrame({"PCC": PCC, "NRMSE": NRMSE}, index = ["RF", "SVM"]).T
PERF = pd.DataFrame([[PCC1, NRMSE1], [PCC2, NRMSE2]], index = ["RF", "SVM"], columns = ["PCC", "NRMSE"])
print("Model performance for 5-fold cross validation = \n", PERF)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, war

No significant correlation!
Model performance for 5-fold cross validation = 
          PCC     NRMSE
RF   0.41224  0.911421
SVM  0.00000  1.000207


In [36]:
# %qtconsole --style monokai
